In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.linalg import hankel
import pyreadr

In [2]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()
scaler = StandardScaler()

In [3]:
L = 3600
stride = 0.5

In [4]:
df_normal = pyreadr.read_r('~/data/tep/TEP_FaultFree_Training.RData')['fault_free_training']
df_normal

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,0.0,500.0,496,0.29325,3640.1,4473.0,9.1949,26.867,42.379,2700.2,...,53.429,29.249,60.773,21.532,40.451,34.064,48.953,48.291,40.812,18.756
249996,0.0,500.0,497,0.29134,3625.7,4506.2,9.2109,26.889,42.291,2700.6,...,53.830,28.975,61.517,21.750,42.762,42.645,51.055,48.589,40.933,19.360
249997,0.0,500.0,498,0.29438,3600.2,4478.3,9.1957,26.820,42.448,2700.3,...,54.163,28.676,61.656,21.487,42.109,39.770,46.770,48.648,41.465,19.344
249998,0.0,500.0,499,0.25269,3683.5,4486.4,9.2832,27.188,42.757,2697.4,...,53.453,24.889,61.564,21.392,39.334,42.274,43.623,48.797,39.835,18.512


In [5]:
df_normal.columns

Index(['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2',
       'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
       'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
       'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20',
       'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26',
       'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
       'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38',
       'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4',
       'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11'],
      dtype='object')

In [6]:
sensors = [col for col in df_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample']]

In [7]:
X_train = pd.DataFrame(index=df_normal.index, columns=sensors, data=scaler.fit_transform(df_normal[sensors]))
Y_train = df_normal.loc[:,'faultNumber']

In [8]:
X_train

,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,-0.003307,0.300365,0.514600,-1.342734,-0.062398,0.294727,-0.098048,-0.252914,0.522187,0.089759,...,-0.489007,0.005770,1.001364,-0.150228,-0.078897,1.424530,0.520012,-0.165135,0.284192,0.224294
1,0.019690,-0.128967,1.218406,0.925081,-0.856781,1.091018,-0.005032,-0.000158,0.522187,-0.068269,...,-1.191640,-0.016944,-1.639731,-0.250199,0.078954,0.153478,-1.208819,-0.195688,0.470947,-0.631770
2,-0.003307,-0.102502,-0.791012,1.114066,-0.128597,-1.224633,0.154425,-0.422648,1.045516,-0.113762,...,0.816186,0.008733,-0.018895,0.308128,0.123494,0.300514,0.070048,-0.180596,0.175098,1.647425
3,-0.023065,-0.073095,0.083646,1.522366,-0.681827,-1.256668,0.287305,0.413107,-0.524469,-0.121743,...,-0.058911,0.028155,-1.159751,0.113845,0.132008,-0.009071,0.428148,-0.110653,0.996079,-0.020296
4,1.411178,0.447396,-0.301407,-0.105001,-0.062398,1.429671,0.008256,0.715676,-0.524469,-0.856813,...,-0.672117,1.368627,-0.467519,-0.508614,-0.596992,1.300426,0.472379,-0.225506,0.744607,0.233859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,1.385266,-0.696510,-0.913413,-1.775532,-0.166424,0.189470,-0.642856,0.983192,0.522187,0.274923,...,-1.159702,1.517422,-0.422496,-1.291404,0.259076,-1.360723,1.028667,0.122366,-0.540488,0.435407
249996,1.323400,-1.119961,-0.066805,-1.588880,-0.062398,-0.213252,-0.589704,1.724856,-0.524469,1.331635,...,-0.305897,1.427223,0.175670,-0.880204,1.772750,1.533121,1.922641,0.232066,-0.316752,0.848067
249997,1.421867,-1.869822,-0.778261,-1.766199,-0.388662,0.505241,-0.629568,-0.542569,0.522187,1.759428,...,0.403124,1.328795,0.287424,-1.376285,1.345044,0.563560,0.100244,0.253785,0.666947,0.837135
249998,0.071514,0.579725,-0.571710,-0.745449,1.351414,1.919344,-1.014920,0.186180,-0.524469,-0.986906,...,-1.108601,0.082143,0.213457,-1.555478,-0.472544,1.408005,-1.238164,0.308634,-2.347016,0.268703


In [9]:
df_attack = pyreadr.read_r('~/data/tep/TEP_Faulty_Training.RData')['faulty_training']
df_attack

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,1,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,1,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,1,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,1,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,1,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,20,500.0,496,0.23419,3655.3,4461.7,9.3448,27.008,42.481,2703.0,...,53.670,23.350,61.061,20.719,40.999,38.653,47.386,47.528,40.212,17.659
4999996,20,500.0,497,0.26704,3647.4,4540.2,9.3546,27.034,42.671,2704.7,...,54.650,26.362,60.020,20.263,41.579,33.624,47.536,47.647,41.199,18.741
4999997,20,500.0,498,0.26543,3630.3,4571.6,9.4089,27.129,42.470,2705.1,...,54.274,26.521,59.824,20.189,41.505,40.967,52.437,47.802,41.302,23.199
4999998,20,500.0,499,0.27671,3655.7,4498.9,9.3781,27.353,42.281,2705.8,...,53.506,26.781,62.818,20.453,40.208,40.957,47.628,48.086,40.510,15.932


In [10]:
X_train_attack = pd.DataFrame(index=df_attack.index, columns=sensors, data=scaler.fit_transform(df_attack[sensors]))
Y_train_attack = df_attack.loc[:,'faultNumber']

In [11]:
X_train_attack

,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,-0.073418,0.236814,0.206726,-0.379428,-0.054130,0.119180,-0.246140,-0.014439,0.134793,-0.095718,...,-0.109079,-0.281921,-0.095175,-0.073796,0.000930,1.441217,0.559507,-0.175924,-0.068961,-0.076644
1,-0.068670,-0.101389,0.455293,0.153737,-0.778257,0.667708,-0.236928,0.087627,0.134793,-0.118746,...,-0.171840,-0.285290,-0.539089,-0.078578,0.019564,0.163216,-1.164070,-0.180640,-0.058873,-0.318689
2,-0.073418,-0.080541,-0.254382,0.198167,-0.114474,-0.927436,-0.221137,-0.082979,0.271950,-0.125375,...,0.007503,-0.281482,-0.266660,-0.051870,0.024822,0.311056,0.110910,-0.178310,-0.074854,0.325736
3,-0.077497,-0.057376,0.054525,0.294159,-0.618777,-0.949503,-0.207979,0.254508,-0.139522,-0.126538,...,-0.070662,-0.278601,-0.458414,-0.061164,0.025827,-0.000222,0.467922,-0.167514,-0.030508,-0.145800
4,0.218614,0.352636,-0.081466,-0.088436,-0.054130,0.900990,-0.235612,0.376688,-0.139522,-0.233653,...,-0.125435,-0.079809,-0.342064,-0.090940,-0.060229,1.316434,0.512018,-0.185243,-0.044091,-0.073939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,-0.181684,-0.196364,-0.399379,-0.070061,0.458794,0.368224,-0.263246,-0.012949,-0.002365,-0.070945,...,-0.123153,-0.345728,-0.295578,-0.201745,0.083197,0.196785,0.402201,-0.174901,-0.173433,-0.228864
4999996,0.037992,-0.379364,0.307594,-0.043183,0.570861,0.967191,-0.240876,0.239608,-0.139522,-0.015934,...,0.063227,-0.198684,-0.436252,-0.242890,0.128042,-1.508459,0.465802,-0.168139,-0.074854,-0.019851
4999997,0.027225,-0.775477,0.590383,0.105741,0.980338,0.333547,-0.235612,-0.545626,0.409108,-0.008956,...,-0.008282,-0.190922,-0.462738,-0.249567,0.122320,0.981421,2.543846,-0.159331,-0.064566,0.841315
4999998,0.102657,-0.187098,-0.064355,0.021269,1.945841,-0.262268,-0.226401,0.025792,-0.413837,-0.088042,...,-0.154343,-0.178229,-0.058148,-0.225746,0.022038,0.978030,0.504810,-0.143193,-0.143669,-0.562474


In [12]:
df_test = pyreadr.read_r('~/data/tep/TEP_Faulty_Testing.RData')['faulty_testing']
X_testt = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_testt = df_test.loc[:,'faultNumber']

In [13]:
attacklabels=hankel.fit(np.asarray(Y_testt),3600,0.5)
attacklabels

array([[ 1,  2,  4, ..., 12, 14, 16],
       [ 1,  2,  4, ..., 12, 14, 16],
       [ 1,  2,  4, ..., 12, 14, 16],
       ...,
       [ 4,  6,  8, ..., 16, 18, 20],
       [ 4,  6,  8, ..., 16, 18, 20],
       [ 4,  6,  8, ..., 16, 18, 20]], dtype=int32)

In [14]:
actual_labels=np.any(attacklabels>0,axis=0).astype(int)
actual_labels

array([1, 1, 1, ..., 1, 1, 1])

In [15]:
X_testt

,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,-0.034241,0.167405,-0.404934,0.321742,0.684188,0.302034,-0.246602,-0.082284,0.144258,-0.097513,...,-0.031842,-0.247820,-0.453629,-0.072551,0.067990,0.690355,0.654306,-0.133671,0.005390,-0.456714
1,-0.029785,-0.528922,0.540153,0.065382,0.429242,0.652800,-0.251653,0.209629,-0.293163,-0.077877,...,-0.192240,-0.251004,-0.368456,-0.068163,0.053357,-0.475039,-0.591255,-0.122140,-0.124873,-0.375910
2,-0.057652,-0.508170,-0.024685,-0.261037,0.112757,-0.303271,-0.273117,-0.245321,-0.293163,-0.090103,...,-0.096525,-0.230720,-0.348628,-0.084930,0.062303,1.301141,-0.798351,-0.123453,-0.058521,0.118027
3,-0.035514,-0.849416,0.084222,-0.144009,0.429242,-0.141856,-0.268067,0.003893,-0.293163,-0.096772,...,-0.114856,-0.246485,-0.205999,-0.102010,0.067383,0.687632,0.694017,-0.125679,-0.105759,-0.256618
4,-0.275282,-0.215344,0.561380,-0.212231,-0.001529,0.081641,-0.220087,-0.369539,-0.001549,-0.286211,...,-0.187265,-0.389143,-0.207914,-0.068242,-0.117996,0.084677,-0.744976,-0.120541,-0.079740,-0.081281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9599995,0.059332,1.255702,0.244813,-0.427920,-0.115815,0.078537,-0.212512,0.428565,-0.001549,-0.399578,...,-0.046507,-0.171253,-0.187973,-0.201749,-0.126108,-0.995943,-0.395689,-0.398952,-0.102728,-0.400331
9599996,-0.028512,1.357154,-0.146512,-0.530778,1.405070,-1.169322,-0.241552,0.018644,0.435871,-0.054537,...,-0.065624,-0.218498,-0.245543,-0.205902,0.127508,0.117361,-1.163010,-0.388734,-0.015409,-0.103226
9599997,-0.034736,0.822228,0.250351,-0.362583,-1.065269,-0.607475,-0.257966,0.166153,0.144258,-0.030332,...,-0.132926,-0.214030,-0.289255,-0.206294,0.109311,-0.709957,-0.332920,-0.370752,-0.126136,-0.143628
9599998,0.243437,-0.538144,0.138675,-0.221153,-0.344387,-0.439852,-0.254178,0.676225,0.144258,-0.238913,...,-0.072302,-0.029880,-0.292523,-0.231914,0.039557,0.599453,-0.185178,-0.360990,-0.101886,-0.236136


In [16]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    idx = np.argmax(fmeas)
    return radiis[indices[idx]]

In [23]:
# def Epasad(X_train,X_train_attack,df_test,clustering,flag):
accuracy_=[]
precision_=[]
recall_=[]
f1_=[]
metrics=[]
for sens in range(len(X_train.columns)):
    X = X_train.iloc[:,sens].values
    X = hankel.fit(X,3600,0.5)
    X=X.T 
    optimal_k = 4
    kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
    kmeans.fit(X)
    print("optimal K",optimal_k)
    radiis_normal = []
    weights=[]
    centers=[]
    clusters_R=[]
    clusters_V=[]
    for i in range(optimal_k):
        cluster_ = X[np.where(kmeans.labels_ == i)[0]]
        r = rank.fit(cluster_)
        print("r : ",r)
        clusters_R.append(r)
        U,Sigma,VT = np.linalg.svd(cluster_)
        V = VT.T
        clusters_V.append(V)
        cluster_ = np.matmul(cluster_,V[:,:r])
        weight,center = meepc.fit(cluster_)
        weights.append(weight)
        centers.append(center)
        var1=np.square(cluster_-center)
        var2=np.matmul(weight,var1.T)
        radiis_normal.append((np.sqrt(var2)))
        


    X_attack = df_attack.iloc[:,sens].values
    X_att = hankel.fit(X_attack,3600,0.5)
    X_att=X_att.T 
    radiis_attack = []
    
    for i in range(optimal_k):
        cluster_ = np.matmul(X_att,clusters_V[i][:,:clusters_R[i]])
        var1=np.square(cluster_-centers[i])
        var2=np.matmul(weights[i],var1.T)
        radiis_attack.append(np.sqrt(var2))

    threshold_clusters = [0]*optimal_k
    
    for i in range(optimal_k):
        threshold_clusters[i] = find_threshold(radiis_normal[i],radiis_attack[i])
    print(threshold_clusters)
        #testing
    X_test=X_testt.iloc[:,sens].values
    Xtest=hankel.fit(X_test,3600,0.5)  
    Xtest=Xtest.T  
    distances=[]
    for i in range(optimal_k):
        Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) 
        var1=np.square(Xtest_cluster-centers[i])
        var2=np.matmul(weights[i],var1.T)
        distance=np.sqrt(var2)
        distances.append(distance)

    stacked_arr=np.vstack(distances)
    distances_arr=np.transpose(stacked_arr)

    # print(distances_arr)
    

    check_anamoly=threshold_clusters-distances_arr    
    y_score = np.min((distances_arr/threshold_clusters),axis=1) 
    print(check_anamoly)
    
    predicted_labels=np.all(check_anamoly<0,axis=1).astype(int)
    print(predicted_labels)
    is_positive=np.any(check_anamoly>0,axis=1)
    
    positive_indices=np.where(is_positive)[0]

    
    accuracy = accuracy_score(actual_labels, predicted_labels)

    precision = precision_score(actual_labels, predicted_labels)
    recall = recall_score(actual_labels, predicted_labels)
    f1 = f1_score(actual_labels, predicted_labels)
    print(accuracy,precision,recall,f1)
    metrics.append([accuracy,precision,recall,f1])
    

optimal K 4
r :  6
r :  23
r :  54
r :  54
[0.7467381782827192, 1.0067272465963946, 1.0075933855408712, 1.0000023346217741]
[[ 0.05996742  0.68581606  0.72823093  0.68190157]
 [ 0.06911111  0.71620037  0.76525679  0.70724474]
 [-0.12892357  0.63820999  0.74223913  0.67923272]
 ...
 [ 0.02828543  0.68155226  0.72768295  0.67588122]
 [ 0.01668159  0.6814512   0.77998989  0.70825021]
 [ 0.06562365  0.67090994  0.69189031  0.68362772]]
[0 0 0 ... 0 0 0]
0.0 0.0 0.0 0.0
optimal K 4
r :  24
r :  46
r :  46
r :  21
[1.0040227638629928, 1.0020396330628767, 0.9984246835791611, 1.000537289474501]
[[0.66653077 0.69270725 0.68804376 0.47891464]
 [0.69386808 0.72115005 0.67894642 0.54827556]
 [0.71704443 0.71779321 0.68719535 0.53294661]
 ...
 [0.67692852 0.68998846 0.64936575 0.45718406]
 [0.68111465 0.70213393 0.67975248 0.46673562]
 [0.63550311 0.64409202 0.61366126 0.43171391]]
[0 0 0 ... 0 0 0]
0.0 0.0 0.0 0.0
optimal K 4
r :  54
r :  22
r :  15
r :  46
[1.0287091698093995, 1.0132914714156729,

In [ ]:
radiis_normal


[array([1.00526909, 0.99287872, 0.90619118, 0.99142286, 0.96614058,
        0.80933188, 0.92120145, 0.98181535, 0.97888837, 0.96774855,
        0.93137482, 0.95053055, 0.95447519, 0.9074258 , 0.94268544,
        0.95299673, 0.94976779, 0.95974537, 0.93145947, 0.96017532,
        0.96022394, 0.96359836, 0.95094251, 0.96930918, 0.96909312,
        0.97842498, 0.98368283, 0.98107542, 0.96818891, 0.95436833,
        0.96463731, 0.93166956, 0.97848144, 0.98229405, 0.97898932,
        0.91641144, 0.96345576, 0.98954333, 0.95730626, 0.99189454,
        0.99077715, 0.99501054, 0.97354979, 0.91614091, 0.94599374,
        0.96880778, 0.94419679, 0.90922373, 0.99489484, 0.98857726,
        0.84871481, 0.96995739, 0.95931767, 0.99095762, 0.98934295,
        0.9568887 , 0.9220347 , 0.99040739, 0.94775719, 0.96684787,
        0.97607478, 0.96740033, 0.99084991, 0.97730802, 0.9948837 ,
        0.87652046, 0.98698587, 0.9785402 , 0.93802657, 0.97688976,
        0.95292264, 0.95161596, 0.97447431, 0.99

In [ ]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    # print(fmeas)
    idx = np.argmax(fmeas)
    # print(indices[idx])
    return radiis[indices[idx]]